# Uncertainty Quantification

## Overview
    we will analyze why we cannot get the right count for some transcripts using the output of salmon. 

## Analyze tools
    we will mainly use dataframe of pandas to analyze the data.

In [2]:
import tsv
import numpy as np
import pandas as pd

## root path

In [3]:
root_path = "../data/poly_mo/"

# data Preprocess

## Poly Truth
    Read the file poly_truth.tsv
    Poly_truth.tsv: true counts for each transcript

In [4]:
# Poly_truth.tsv: true counts for each transcript
poly_truth = open(root_path+"poly_truth.tsv")
lines = poly_truth.readlines()
poly_truth.close()
# print l
count = 0
poly_truth = []
for line in lines:
    line = line[:-1]
    l = line.split('\t')
    poly_truth.append(l)

df_poly_truth = pd.DataFrame.from_records(poly_truth[1:], columns=poly_truth[0])

In [5]:
df_poly_truth['transcript_id']=df_poly_truth['transcript_id'].astype(str)
df_poly_truth['count']=df_poly_truth['count'].astype(int)

In [6]:
df_poly_truth.describe()

,count
count,2.695900e+04
mean,1.483678e+03
std,1.504054e+04
min,1.000000e+00
25%,1.700000e+01
50%,1.420000e+02
75%,7.350000e+02
max,1.316896e+06


In [7]:
df_poly_truth.head(5)

,transcript_id,count
0,ENST00000608495,1
1,ENST00000382369,55
2,ENST00000360321,54
3,ENST00000400269,92
4,ENST00000382352,2653


In [8]:
truth_id = df_poly_truth.transcript_id

## Quant_bootstraps
    Read the file quant_bootstraps.tsv
    Quant_bootstraps.tsv :containing the matrix of bootstrap experiments containing the final count for each transcript in each round of bootstrapping with a row be a bootstrap output and columns be list of transcripts. 

In [10]:
# Quant_bootstraps.tsv :containing the matrix of bootstrap experiments 
# containing the final count for each transcript in each round of bootstrapping 
# with a row be a bootstrap output and columns be list of transcripts. 

quant_bootstraps = tsv.TsvReader(open(root_path+"quant_bootstraps.tsv"))
count = 0
quant_boot = []
for parts in quant_bootstraps:
    quant_boot.append(parts)
#     print len(parts)
print(len(quant_boot))

201


In [11]:
df_quant_boot = pd.DataFrame.from_records(quant_boot[1:], columns=quant_boot[0])
# print(len(quant_boot[1,:]))

In [12]:
df_quant_boot.describe()

,ENST00000382410,ENST00000382398,ENST00000382388,ENST00000334391,ENST00000246105,ENST00000382376,ENST00000608495,ENST00000382369,ENST00000360321,ENST00000400269,...,ENST00000361739,ENST00000361851,ENST00000361899,ENST00000362079,ENST00000361227,ENST00000361335,ENST00000361381,ENST00000361567,ENST00000361681,ENST00000361789
count,200,200,200,200,200,200,200,200,200,200,...,200,200,200,200,200,200,200,200,200,200
unique,1,1,1,1,1,1,1,200,200,200,...,190,148,187,187,172,165,191,180,185,187
top,0.0,0.0,0.0,0.0,0.0,0.0,0.0,31.137669328,17.2697468599,61.2402575227,...,340011.0,8220.0,226020.0,350241.0,30662.0,22438.0,415926.0,210106.0,72774.0,155444.0
freq,200,200,200,200,200,200,200,1,1,1,...,3,4,2,2,3,3,2,3,2,3


In [13]:
id_qb = list(df_quant_boot.columns)

### Sort the quant_boot Results
#### Attention: there are some ids in truth_id but not in quant_boot

In [14]:
sort_qb = []
use_id = []
for id in truth_id:
    try:
        listed = list(df_quant_boot[id])        
    except KeyError:
#         print('has No '+id) # there are some ids in truth_id but not in quant_boot
        pass
    else:
        use_id.append(id)
        listed.sort()
        sort_qb.append(listed)

### reverse sort_qb

In [15]:
sort_qb = list(map(list,zip(*sort_qb)))

### transfer to dataframe

In [16]:
df_qb_sorted = pd.DataFrame.from_records(sort_qb, columns=use_id)

## Filter the False

### find the value of 2.5% & 97.5% and the false transcript is out of this range

In [17]:
df_poly_truth = df_poly_truth.set_index(['transcript_id'])

In [18]:
sum = len(sort_qb)
print(int(sum*0.025))
print(int(sum*0.975))
percent2dot5 = df_qb_sorted.loc[int(sum*0.025)-1]
percent97dot5 = df_qb_sorted.loc[int(sum*0.975)-1]

5
195


## divide the transcript_id into two group
    if the true_count of the id is in the range(2.5%,97.5%)，we will set it in the true_id group.
    if the true_coutn of the id is out of the range(2.5%,97.5%), we set it in the false_id group.

In [19]:
true_id = []
false_id = []
for id in use_id:
    down = float(percent2dot5[id])
    up = float(percent97dot5[id])
    true_count = df_poly_truth.loc[id]
    true_count = float(true_count)
    if true_count>down and true_count<up:
        true_id.append(id)
    else:
        false_id.append(id)

### directly set the transcript_id whose true_count is zeros into the true_id group

#### get the different set of id from quant_boot and poly_truth.
    This set include the transcripts_id whose true count is zero.

In [20]:
extend_true = list(set(id_qb).difference(set(truth_id)))

#### get the extended true_id

In [21]:
true_id.extend(extend_true)

### concatenate the true and false id
    we put the true_id and false_id in the same list in sequence, and then we will add label for them.

In [22]:
# concatenate the true and false id in one list
all_id = list(true_id)
all_id.extend(false_id)

### add label for the list
    set label for every transcript_id(success(true_id,set as 1),fail(false_id,set as 0))
    And them we will merge this labeled list with list of properties in order to get a list which include both properties and label of every transcript.

In [23]:
# add label for the id
label = []
for i in range(len(true_id)):
    label.append(1)
for i in range(len(false_id)):
    label.append(0)
    
labeled_id = [all_id,label]
labeled = list(map(list,zip(*labeled_id)))

## Read Quant.sf
    Read the quant.sf file.
    Quant.sf :estimated quantifications for each transcript

In [24]:
# Quant.sf :estimated quantifications for each transcript
quant_file = open(root_path+"quant.sf")
lines = quant_file.readlines()
quant_file.close()
count = 0
quant = []
for line in lines:
    line = line[:-1]
    l = line.split('\t')
    quant.append(l)

In [25]:
df_quant = pd.DataFrame.from_records(quant[1:], columns=quant[0])

In [26]:
df_quant.Name = df_quant.Name.astype(str)
df_quant.Length = df_quant.Length.astype(int)
df_quant.EffectiveLength = df_quant.EffectiveLength.astype(float)
df_quant.TPM = df_quant.TPM.astype(float)
df_quant.NumReads = df_quant.NumReads.astype(float)

## Merge quant.sf and labeled_id to get the useful data for training
    labeled_id is a list of transcript_id togather with label(success(true，set as 1) or fail(flase,set as 0))  
    And we will add the label with the protery from quant.sf in order to analyze the properties of different label.
    Then it will be easy for us to analyze the relation between properties and label and the difference between group of different label.

#### merge the data

In [27]:
df_labeled_id = pd.DataFrame.from_records(labeled, columns=['Name','label'])
df_labeled_id.Name = df_labeled_id.Name.astype(str)
df_labeled = df_labeled_id.merge(df_quant, on='Name')

#### put label as the last property

In [28]:
label = df_labeled.pop('label')
df_labeled.insert(5,'label',label)

#### set "data = df_labeled" for easy processing the data below 

In [29]:
data = df_labeled

## Analyze the difference between true & false data

### we can easily found that the mean value of TPM & NumReads has great difference between false and true

In [30]:
data.groupby([data.label]).mean()

,Length,EffectiveLength,TPM,NumReads,label
label,,,,,
0,2445.007350,2245.894343,52.556885,1784.370671,0.0
1,1905.694197,1706.993431,0.841197,20.235059,1.0


### the max value of TPM & NumReads alse has great difference between false and true

In [31]:
data.groupby([data.label]).max()

,Name,Length,EffectiveLength,TPM,NumReads
label,,,,,
0,ENST00000610278,101518,101318.991,23356.420222,1.109005e+06
1,ENST00000610279,109224,109024.991,10710.459004,3.769085e+04


In [32]:
data.groupby([data.label]).min()

,Name,Length,EffectiveLength,TPM,NumReads
label,,,,,
0,ENST00000000233,158,10.987,0.0,0.0
1,ENST00000000412,21,9.784,0.0,0.0


In [33]:
data.groupby([data.label]).std()

,Length,EffectiveLength,TPM,NumReads
label,,,,
0,2380.728560,2380.669848,448.368497,15740.352178
1,2055.905694,2055.526264,45.873037,235.234059


## Eq_classes.txt
    Eq_classes.txt: list of equivalence classes and their information

In [25]:
file = open(root_path+"eq_classes.txt")
lines = file.readlines()
file.close()

In [26]:
counts_transcript = lines[0]
counts_transcript = int(counts_transcript[:-1])
counts_class = lines[1]
counts_class = int(counts_class[:-1])

In [27]:
lines = lines[2:]

### get the sequence and order of id

In [28]:
seq_id=[]
for id in lines[0:counts_transcript]:
    seq_id.append(id[:-1])

In [29]:
lines = lines[counts_transcript:]

In [33]:
lines[0]

'1\t61539\t385\n'

In [34]:
list_class = []
for line in lines:
    li = line[:-1].split('\t')
    list_class.append(li)

In [35]:
list_class[0]

['1', '61539', '385']

In [36]:
list_class[-1]

['5', '83713', '83714', '83715', '83716', '83717', '128']